# Build Model Workbook

In [3]:
import pandas as pd
import os
import matplotlib.pyplot as plt
import numpy as np

Load Dataset

In [4]:
data_dir = 'data'
processed_data = 'processed_data'

In [5]:
dataset = pd.read_csv(os.path.join(processed_data, 'dataset.csv'))

In [6]:
dataset.head(10)

,center_image,left_image,right_image,steering_angle,throttle,break,speed,folder
0,center_2019_05_24_13_42_51_893.jpg,left_2019_05_24_13_42_51_893.jpg,right_2019_05_24_13_42_51_893.jpg,0.0,0.000000,0.0,24.62923,driving_data_2
1,center_2019_05_24_13_42_51_970.jpg,left_2019_05_24_13_42_51_970.jpg,right_2019_05_24_13_42_51_970.jpg,0.0,0.000000,0.0,24.43121,driving_data_2
2,center_2019_05_24_13_42_52_041.jpg,left_2019_05_24_13_42_52_041.jpg,right_2019_05_24_13_42_52_041.jpg,0.0,0.000000,0.0,24.23477,driving_data_2
3,center_2019_05_24_13_42_52_111.jpg,left_2019_05_24_13_42_52_111.jpg,right_2019_05_24_13_42_52_111.jpg,0.0,0.000000,0.0,24.08849,driving_data_2
4,center_2019_05_24_13_42_52_179.jpg,left_2019_05_24_13_42_52_179.jpg,right_2019_05_24_13_42_52_179.jpg,0.0,0.000000,0.0,23.94309,driving_data_2
5,center_2019_05_24_13_42_52_246.jpg,left_2019_05_24_13_42_52_246.jpg,right_2019_05_24_13_42_52_246.jpg,0.0,0.000000,0.0,23.75058,driving_data_2
6,center_2019_05_24_13_42_52_316.jpg,left_2019_05_24_13_42_52_316.jpg,right_2019_05_24_13_42_52_316.jpg,0.0,0.000000,0.0,23.60721,driving_data_2
7,center_2019_05_24_13_42_52_384.jpg,left_2019_05_24_13_42_52_384.jpg,right_2019_05_24_13_42_52_384.jpg,0.0,0.118662,0.0,23.44914,driving_data_2
8,center_2019_05_24_13_42_52_451.jpg,left_2019_05_24_13_42_52_451.jpg,right_2019_05_24_13_42_52_451.jpg,0.0,0.318925,0.0,23.45194,driving_data_2
9,center_2019_05_24_13_42_52_526.jpg,left_2019_05_24_13_42_52_526.jpg,right_2019_05_24_13_42_52_526.jpg,0.0,0.547578,0.0,23.67355,driving_data_2


# Build Simple Dataset

center image only with steering angles

In [7]:
x_set = []
y_set = []
for rn, row in dataset.iterrows():
    path = os.path.join('.', data_dir, row['folder'], 'IMG', row['center_image'])
    image = plt.imread(path)
    x_set.append(image)
    y_set.append(row['steering_angle'])

In [8]:
len(x_set)

7921

In [9]:
dataset['folder'].unique()

array(['driving_data_2', 'driving_data_1', 'driving_data_3'], dtype=object)

In [10]:
x_set[0].shape

(160, 320, 3)

In [11]:
x_train = np.array(x_set)
y_train = np.array(y_set)

In [12]:
x_train.shape

(7921, 160, 320, 3)

In [13]:
y_train.shape

(7921,)

# Simple Model

In [14]:
from keras.models import Sequential
from keras.layers import Flatten, Dense

In [17]:
model = Sequential()
model.add(Flatten(input_shape=(160, 320, 3)))
model.add(Dense(1))

model.compile(loss='mse', optimizer='adam')
model.fit(x_train, y_train, validation_split=0.2, shuffle=True)

Train on 6336 samples, validate on 1585 samples
Epoch 1/10
6336/6336 [==============================] - 2s 314us/step - loss: 8032748.8340 - val_loss: 3967.8531
Epoch 2/10
6336/6336 [==============================] - 2s 310us/step - loss: 4053.8818 - val_loss: 2785.3772
Epoch 3/10
6336/6336 [==============================] - 2s 312us/step - loss: 3034.5205 - val_loss: 1446.7063
Epoch 4/10
6336/6336 [==============================] - 2s 310us/step - loss: 2922.3319 - val_loss: 2357.3385
Epoch 5/10
6336/6336 [==============================] - 2s 311us/step - loss: 2493.4174 - val_loss: 3453.5532
Epoch 6/10
6336/6336 [==============================] - 2s 309us/step - loss: 3728.5733 - val_loss: 3806.1823
Epoch 7/10
6336/6336 [==============================] - 2s 314us/step - loss: 1947.6531 - val_loss: 6452.4513
Epoch 8/10
6336/6336 [==============================] - 2s 309us/step - loss: 4798.5708 - val_loss: 867.7255
Epoch 9/10
6336/6336 [==============================] - 2s 314us/step 

In [19]:
model.save('model.h5')

# Advanced Model

In [15]:
from keras.layers import Lambda, Convolution2D, Cropping2D

In [16]:
adv_model = Sequential()
adv_model.add(Lambda(lambda x: x/255.0 - 0.5, input_shape=(160,320,3)))
adv_model.add(Cropping2D(cropping=((70,25),(0,0))))
adv_model.add(Convolution2D(24,5,5,subsample=(2,2),activation="relu"))
adv_model.add(Convolution2D(36,5,5,subsample=(2,2),activation="relu"))
adv_model.add(Convolution2D(48,5,5,subsample=(2,2),activation="relu"))
adv_model.add(Convolution2D(64,3,3,activation="relu"))
adv_model.add(Convolution2D(64,3,3,activation="relu"))
adv_model.add(Flatten())
adv_model.add(Dense(100))
adv_model.add(Dense(50))
adv_model.add(Dense(10))
adv_model.add(Dense(1))

/root/miniconda3/envs/carnd-term1/lib/python3.5/site-packages/ipykernel_launcher.py:4: UserWarning: Update your `Conv2D` call to the Keras 2 API: `Conv2D(24, (5, 5), strides=(2, 2), activation="relu")`
  after removing the cwd from sys.path.
/root/miniconda3/envs/carnd-term1/lib/python3.5/site-packages/ipykernel_launcher.py:5: UserWarning: Update your `Conv2D` call to the Keras 2 API: `Conv2D(36, (5, 5), strides=(2, 2), activation="relu")`
  """
/root/miniconda3/envs/carnd-term1/lib/python3.5/site-packages/ipykernel_launcher.py:6: UserWarning: Update your `Conv2D` call to the Keras 2 API: `Conv2D(48, (5, 5), strides=(2, 2), activation="relu")`
  
/root/miniconda3/envs/carnd-term1/lib/python3.5/site-packages/ipykernel_launcher.py:7: UserWarning: Update your `Conv2D` call to the Keras 2 API: `Conv2D(64, (3, 3), activation="relu")`
  import sys
/root/miniconda3/envs/carnd-term1/lib/python3.5/site-packages/ipykernel_launcher.py:8: UserWarning: Update your `Conv2D` call to the Keras 2 API: 

In [19]:
adv_model.compile(loss='mse', optimizer='adam')
adv_model.fit(x_train, y_train, epochs=5, validation_split=0.2, shuffle=True)

Train on 6336 samples, validate on 1585 samples
Epoch 1/5
6336/6336 [==============================] - 35s 6ms/step - loss: 0.0339 - val_loss: 0.1708
Epoch 2/5
6336/6336 [==============================] - 35s 6ms/step - loss: 0.0292 - val_loss: 0.1808
Epoch 3/5
6336/6336 [==============================] - 36s 6ms/step - loss: 0.0267 - val_loss: 0.1745
Epoch 4/5
6336/6336 [==============================] - 36s 6ms/step - loss: 0.0247 - val_loss: 0.1685
Epoch 5/5
6336/6336 [==============================] - 36s 6ms/step - loss: 0.0208 - val_loss: 0.1799


In [20]:
adv_model.save('adv_model.h5')